# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [55]:
# remove 
import problem_unittests as tests

In [4]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
    int_to_vocab = dict(enumerate(vocab,1))
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

<font color='red'>HS: in sentiment_RNN we removed all punctuations, here we are tokenizing them so that our network takes and returns punctuations as well, similar to the character level </font>

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punc_to_token = {'.':'|period|',
                     ',':'|comma|',
                     '"':'|quotation_mark|',
                     ';':'|semicolon|',
                     '!':'|exclamation_mark|',
                     '?':'|question_mark|',
                     '(':'|left_parentheses|',
                     ')':'|right_parentheses|',
                     '--':'|dash|',
                     '\n':'|return|'}
    return punc_to_token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

/home/ubuntu/anaconda3/envs/tv-scripts-project/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [3]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    Input = tf.placeholder(tf.int32, [None, None], name='input')
    Targets = tf.placeholder(tf.int32, [None, None], name='targets')
    LearningRate = tf.placeholder(tf.float32, name='learning_rate')
    # TODO: Implement Function
    return Input, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [4]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    #hs rnn_size is the number of neurons in each hidden layer
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    #hs we have 2 hidden layers
    Cell = tf.contrib.rnn.MultiRNNCell([lstm] * 2)
    initial_state = Cell.zero_state(batch_size, tf.float32)
    InitialState = tf.identity(initial_state, name='initial_state')
    return Cell, InitialState 


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [5]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embedded = tf.nn.embedding_lookup(embedding, input_data)
    return embedded


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [6]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    #Outputs, final_state = tf.nn.dynamic_rnn(cell, inputs,
                                             #initial_state=initial_state)
    #hs you either have to give the initial state or its type
    Outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
                                             
    FinalState = tf.identity(final_state, name='final_state')
    return Outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

<font color='red'>HS: Note that in sentiment_rnn, we only cared about the last ouput. Therfore, we took the output of the last element in a sequence, and pass it to a fullyConnected layer which concatenates it and applies any activation we want to it. We also took a single output (i.e. set the number of outputs to linear layer to 1) because we had binary classification. 
<br><br>
In this project, we care about all the outputs (the output for each step of the sequence) and we have multilabel classification. So we passed the entire output vector to our linearLayer and we set the vocab-size as the number of outputs we expect from it.
<br>
So if we have 128 sequences (i.e. batch size of 128) of length 5, as our input tensor, and we have 27 words in our vocab. Then the output of linearLayer (i.e. Logits) is [128, 5, 27]  
</font>

In [7]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    input_embedded = get_embed(input_data, vocab_size, embed_dim)
    outputs, FinalState = build_rnn(cell, input_embedded)
    #hs take the last output and apply sigmoid to it
    #Logits = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    Logits = tf.contrib.layers.fully_connected(outputs[:,:], vocab_size, activation_fn=tf.sigmoid)
    return Logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [8]:
batch_size = 124
seq_length = 27
number_of_batches = 100
Batches = np.zeros((number_of_batches, 2, batch_size, seq_length), np.int32)
print(Batches.shape)

#(number of batches, 2, batch size, sequence length)

(100, 2, 124, 27)


In [8]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    words_per_batch = batch_size * seq_length
    number_of_batches = len(int_text)//words_per_batch
    
    #hs convert list to np array
    arr = np.array(int_text)
    
    # Keep only enough characters to make full batches
    arr = arr[:number_of_batches * words_per_batch]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    '''
    hs above is saying nb of rows is batch_size and the second dimention should be whatever is 
    requiredto give batch_size rows.
    '''
    Batches = np.zeros((number_of_batches, 2, batch_size, seq_length), np.int32)
    batchNb = 0
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]

        # The targets, shifted by one
        '''
        hs: y[:, :-1] means all rows of y, and all the columns starting from start till 
        some place holder (i.e. till as much as we feed you which will be 1 less than x columns).
        And we fill this wtih x[:, 1:] which is all rows of x but columns starting from 1 (so shifted)
        by 1 colomn to the right. And then the last element of y, becomes first element of x. 
        (The tradition is to wrap around x like this)
        '''
        y = np.zeros_like(x)
        #y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        y[:, :-1] = x[:, 1:]
        if (n+seq_length) < arr.shape[1]: 
            y[:, -1] =  arr[:, n+seq_length]
        
        Batches[batchNb,0,:,:] = x
        Batches[batchNb,1,:,:] = y
        batchNb = batchNb + 1
    
    #last target of the last batch should be the first input of the first batch
    #Batches[batchNb-1,1,:,-1] = Batches[0,0,:,0]
    Batches[batchNb-1,-1,-1,-1] = Batches[0,0,0,0]
    return Batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [40]:
# Number of Epochs
num_epochs = 1000
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 500
# Sequence Length
seq_length = 100
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './saved'

### Build the Graph
Build the graph using the neural network you implemented.

In [41]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [42]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/5   train_loss = 8.822
Epoch   0 Batch    1/5   train_loss = 8.815
Epoch   0 Batch    2/5   train_loss = 8.800
Epoch   0 Batch    3/5   train_loss = 8.761
Epoch   0 Batch    4/5   train_loss = 8.705
Epoch   1 Batch    0/5   train_loss = 8.638
Epoch   1 Batch    1/5   train_loss = 8.570
Epoch   1 Batch    2/5   train_loss = 8.503
Epoch   1 Batch    3/5   train_loss = 8.437
Epoch   1 Batch    4/5   train_loss = 8.383
Epoch   2 Batch    0/5   train_loss = 8.330
Epoch   2 Batch    1/5   train_loss = 8.284
Epoch   2 Batch    2/5   train_loss = 8.252
Epoch   2 Batch    3/5   train_loss = 8.227
Epoch   2 Batch    4/5   train_loss = 8.214
Epoch   3 Batch    0/5   train_loss = 8.197
Epoch   3 Batch    1/5   train_loss = 8.182
Epoch   3 Batch    2/5   train_loss = 8.171
Epoch   3 Batch    3/5   train_loss = 8.164
Epoch   3 Batch    4/5   train_loss = 8.163
Epoch   4 Batch    0/5   train_loss = 8.159
Epoch   4 Batch    1/5   train_loss = 8.151
Epoch   4 Batch    2/5   train_l

Epoch  37 Batch    2/5   train_loss = 8.122
Epoch  37 Batch    3/5   train_loss = 8.123
Epoch  37 Batch    4/5   train_loss = 8.125
Epoch  38 Batch    0/5   train_loss = 8.128
Epoch  38 Batch    1/5   train_loss = 8.125
Epoch  38 Batch    2/5   train_loss = 8.122
Epoch  38 Batch    3/5   train_loss = 8.123
Epoch  38 Batch    4/5   train_loss = 8.125
Epoch  39 Batch    0/5   train_loss = 8.128
Epoch  39 Batch    1/5   train_loss = 8.125
Epoch  39 Batch    2/5   train_loss = 8.122
Epoch  39 Batch    3/5   train_loss = 8.123
Epoch  39 Batch    4/5   train_loss = 8.125
Epoch  40 Batch    0/5   train_loss = 8.128
Epoch  40 Batch    1/5   train_loss = 8.125
Epoch  40 Batch    2/5   train_loss = 8.122
Epoch  40 Batch    3/5   train_loss = 8.123
Epoch  40 Batch    4/5   train_loss = 8.125
Epoch  41 Batch    0/5   train_loss = 8.128
Epoch  41 Batch    1/5   train_loss = 8.125
Epoch  41 Batch    2/5   train_loss = 8.122
Epoch  41 Batch    3/5   train_loss = 8.123
Epoch  41 Batch    4/5   train_l

Epoch  74 Batch    4/5   train_loss = 8.124
Epoch  75 Batch    0/5   train_loss = 8.127
Epoch  75 Batch    1/5   train_loss = 8.124
Epoch  75 Batch    2/5   train_loss = 8.122
Epoch  75 Batch    3/5   train_loss = 8.122
Epoch  75 Batch    4/5   train_loss = 8.124
Epoch  76 Batch    0/5   train_loss = 8.127
Epoch  76 Batch    1/5   train_loss = 8.124
Epoch  76 Batch    2/5   train_loss = 8.122
Epoch  76 Batch    3/5   train_loss = 8.122
Epoch  76 Batch    4/5   train_loss = 8.124
Epoch  77 Batch    0/5   train_loss = 8.127
Epoch  77 Batch    1/5   train_loss = 8.124
Epoch  77 Batch    2/5   train_loss = 8.122
Epoch  77 Batch    3/5   train_loss = 8.122
Epoch  77 Batch    4/5   train_loss = 8.124
Epoch  78 Batch    0/5   train_loss = 8.127
Epoch  78 Batch    1/5   train_loss = 8.124
Epoch  78 Batch    2/5   train_loss = 8.122
Epoch  78 Batch    3/5   train_loss = 8.122
Epoch  78 Batch    4/5   train_loss = 8.124
Epoch  79 Batch    0/5   train_loss = 8.127
Epoch  79 Batch    1/5   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [34]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

In [29]:
with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, "./saveFin/finMod")

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [30]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [31]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    #vals, indices = tf.nn.top_k(probabilities,1)
   # vals, indices = probabilities.data.topk(1)#tf.nn.top_k(probabilities,1)
    #vals, indices = tf.nn.top_k(probabilities,1)
    word_index = probabilities.argmax(axis=0)
    return int_to_vocab[word_index]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [35]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: goes plant christmas christmas christmas answered place place hubub located located located foolin' yell yell yell yell peppers peppers prison prison prison plaintive catty catty catty barney's barney's english english english english whee whee whee eaters eaters eaters eaters walked walked scrubbing scrubbing facebook facebook clone clone raise lost raise raise radiation lone lone tried tried-------- known known known known feisty feisty town's guttural frog slender slender slender angrily angrily address protesting soft soft add womb womb hangs hangs alfalfa bringin' bringin' remorseful remorseful remorseful mr teenage_barney: mr kitchen seat seat seat c'mon unless unless unless four-star four-star four-star sending sending sending ringing shells bloodball bloodball bloodball movies movies movies movies hollowed-out don't don't don't charter kisser kisser lloyd: lloyd: conspiracy conspiracy hollowed-out murmur murmur murmur barber ruuuule ruuuule sour sour sour hoagie ho

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.